In [1]:
# Импорт необходимых библиотек

import pandas as pd
import cv2, os
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import os

In [37]:
# Экстракция кадров содержащих лица

cam = cv2.VideoCapture('come.mp4')

face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

face_id = '0001'
count = 0

M = np.float32([[0.5, 0, 0], [0, 0.5, 0]])
size = (2688, 1520)

while cam.isOpened():
    ret, img = cam.read()
    if ret == True:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 
                                               scaleFactor=1.2, 
                                               minNeighbors=5, 
                                               minSize=(30, 30))
        for (x,y,w,h) in faces:     
            cv2.rectangle(img, (x, y), (x+w,y+h), (255, 0, 0), 2)
            count += 1     
            # Сохраняем лицо
            cv2.imwrite('face/user.' + str(face_id) + '.' + str(count) + '.jpg', gray[y:y+h,x:x+w]) 
            if count >= 100: # Если сохранили 30 изображений выход.
                break
    else:
        break
    cv2.imshow('camera', img)
        
    k = cv2.waitKey(10) & 0xff  # 'ESC' для Выхода


print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()


 [INFO] Exiting Program and cleanup stuff


In [38]:
def assure_path_exists(path):
    dir = os.path.dirname(path)
    if not os.path.exists(dir):
        os.makedirs(dir)

In [39]:
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [40]:
detector = cv2.CascadeClassifier(r"/Users/Daniil/Downloads/opencv/sources/data/haarcascades_cuda/haarcascade_frontalface_default.xml")

In [41]:
# Create method to get the images and label data
def getImagesAndLabels(path):

    # Get all file path
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)] 
    
    # Initialize empty face sample
    faceSamples=[]
    
    # Initialize empty id
    ids = []

    # Loop all the file path
    for imagePath in imagePaths:

        # Get the image and convert it to grayscale
        PIL_img = Image.open(imagePath).convert('L')

        # PIL image to numpy array
        img_numpy = np.array(PIL_img,'uint8')

        # Get the image id
        id = int(os.path.split(imagePath)[-1].split(".")[1])

        # Get the face from the training images
        faces = detector.detectMultiScale(img_numpy)

        # Loop for each face, append to their respective ID
        for (x,y,w,h) in faces:

            # Add the image to face samples
            faceSamples.append(img_numpy[y:y+h,x:x+w])

            # Add the ID to IDs
            ids.append(id)

    # Pass the face array and IDs array
    return faceSamples, ids

In [42]:
faces,ids = getImagesAndLabels('face/Andrey')

In [43]:
# Train the model using the faces and IDs
recognizer.train(faces, np.array(ids))

In [44]:
# Save the model into trainer.yml
assure_path_exists('trainer/')
recognizer.save('trainer/trainer.yml')

In [57]:
import pandas as pd
from datetime import datetime, date, time

cam = cv2.VideoCapture('come.mp4')
cam.set(3, 2688)  # set video width
cam.set(4, 1520)  # set video height

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer/trainer.yml")
font = cv2.FONT_HERSHEY_SIMPLEX

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

t = pd.read_csv('табель опозданий.csv')
current_date = date.today().strftime("%Y-%m-%d")
d = date(2023, 5, 23)
time_n = time(10, 0)
time_work = datetime.combine(date.today(), time_n)


names = ['none', 'Andrey', 'Artur', 'Daniil', 'Evgeniy', 'Ruslan', 'Yra']
id = [0, 1, 2, 3, 4, 5, 6]
while cam.isOpened():
    ret, img = cam.read()
    if ret == True:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 
                                               scaleFactor=1.2, 
                                               minNeighbors=5)
    
        for (x, y, w, h) in faces:
            
            cv2.rectangle(img, (x, y), (x+w,y+h), (255, 0, 0), 2)
            id, confidence = recognizer.predict(gray[y:y + h, x:x + w])
            print(id, confidence)
            # Проверяем что лицо распознано
            if (confidence > 130):
                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
                if len(t.loc[(t['Сотрудник'] == id) & (t['Дата'] == current_date)]) == 0:
                    
                    new_index = len(t)
                    t.loc[new_index, ['Сотрудник']] = id
                    t.loc[new_index, ['Дата']] = date.today().strftime("%Y-%m-%d")
                    t.loc[new_index, ['Время прихода']] = datetime.now()
                    t.loc[new_index, ['Время опоздания']] = datetime.now() - time_work
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))
    
            cv2.putText(img, str(id), (x + 5, y - 5), font, 1, (255, 255, 255), 2)
            cv2.putText(img, str(confidence), (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)
    else:
        t.to_csv('табель опозданий.csv', index=False)
        break
    
    cv2.imshow('camera', img)

    k = cv2.waitKey(10) & 0xff  # 'ESC' для Выхода
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()

3 59.82468146967732
3 60.75353078639252
3 60.28554095449236
3 62.16737142527301
3 61.119844537502786
3 61.59750725485572
3 61.419173321491655
3 62.10891787941901
3 62.25236851819183
3 59.99238456045609
3 60.75087345735025
3 60.947480209759576
3 61.16373858337988
3 60.60159315603377
3 61.334123774239906
3 61.417225877768715
3 61.83022755583733
3 61.09685283388446
3 60.93476678110711
3 61.386716305014296
3 61.150750681911106
3 61.717014528398764
3 61.721721948593995
3 61.682733093121655
3 61.651702166093784
3 61.60298736730751
3 61.96650173559165
3 61.615097089407364
3 61.59878464579209
3 62.26770141059466
3 62.21593832002582
3 61.96969075898976
3 62.56392839447421
3 62.46175413646519
3 62.254556624462424
3 61.02536161338345
3 62.86712120275525
4 113.12788822650764
3 58.81242614362016
3 59.06786824438952
3 58.665151674227616
3 56.41722453859941
3 57.96575848226726
4 77.3319530093162
6 91.85344396200149
6 94.87159703377742
6 90.73735469614122
6 80.55897475044044
3 57.367327424960784
3 58.

In [56]:
t = pd.read_csv('табель опозданий.csv')
t

,Сотрудник,Дата,Время прихода,Время опоздания
0,Daniil,2023-10-27,2023-10-27 11:00:08.130172,1:00:08.130172
1,Ruslan,2023-10-27,2023-10-27 11:00:10.704744,1:00:10.704744
